In [1]:
from cgra import *
from kernels import *
kernel_name = "mmul_os"
# Input data
kernel_clear_memory(kernel_name)
kernel_add_memory_region(kernel_name, 100, [10, 20, 30, 40, 50, 60,70,80,90,100,110,120,130,140,150,160,170,180,190,200])
kernel_add_memory_region(kernel_name, 1000, [100,104,108,112, 4,1, 1, 2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16,17,18,19])
# Set load and write addrs
load_addrs = [1000, 1016, 1020, 1000+(4+2+4*3)*4]
store_addrs = [0, 16, 32, 48]
run(kernel_name, pr=["ROUT","INST"], load_addrs=load_addrs, store_addrs=store_addrs)

Instr =  0 ( 0 )
[ 100,    4,    1,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  1 ( 1 )
[ 100,    4,    0,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  2 ( 2 )
[ 100,    4,   18,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  3 ( 3 )
[   0,    0,    0,    0]    [
[   0,    0,    0,    0]    [
[   0,    0,    0,    0]    [
[   0,    0,    0,    0]    [
-------
Instr =  4 ( 4 )
[ 100,    0,    0,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  5 ( 5 )
[ 100,   15,    0,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  6 ( 6 )
[   1,    5,    9,   13]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0

In [3]:
from sat_to_csv import *
from kernels import *
from cgra import *
kernel_name = "mmul_os"
convert( kernel_name+"/out.sat", kernel_name+"/instructions.csv" )

In [4]:
from cgra import *
from kernels import *
kernel_name = "mmul_os"
# ----------- Op 4 - v2 ---------
# Input data
# 8x8x4
kernel_clear_memory(kernel_name)
# Add B
B = list(range(10, 321, 10))
baseB = 2000
kernel_add_memory_region(kernel_name, baseB, B)
# Add auxiliar address and counters
baseA = 5000
kernel_add_memory_region(kernel_name, baseA, [baseB,baseB+4,baseB+8,baseB+12, 8,2])
# Add A
A = list(range(1, 65, 1))
kernel_add_memory_region(kernel_name, baseA+28, A)
# Set load and write addrs
load_addrs = [baseA, baseA+16, baseA+20, baseA+(4+2+8*3)*4] # Last addrs is 1000+4*(4+2+nColsA*3)
store_addrs = [0, 16, 32, 48] # Each one is one row, so [0, 4*nColsC, 4*nColsC*2, 4*nColsC*3]
run(kernel_name, pr=["ROUT","INST"], load_addrs=load_addrs, store_addrs=store_addrs)

Instr =  0 ( 0 )
[2000,    8,    2,    0]    [LWD R2  4, LWD R1  24, LWD R1  36, NOP ]    
[2004,    0,    0,    0]    [LWD R2  4, NOP , NOP , NOP ]    
[2008,    0,    0,    0]    [LWD R2  4, NOP , NOP , NOP ]    
[2012,    0,    0,    0]    [LWD R2  12, NOP , NOP , NOP ]    
-------
Instr =  1 ( 1 )
[2000,    8,    0,    0]    [NOP , NOP , SADD R2  ZERO  ZERO, NOP ]    
[2004,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
[2008,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
[2012,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
-------
Instr =  2 ( 2 )
[2000,    8,   18,    0]    [NOP , NOP , BEQ R2  R1  18, NOP ]    
[2004,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
[2008,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
[2012,    0,    0,    0]    [NOP , NOP , NOP , NOP ]    
-------
Instr =  3 ( 3 )
[   0,    0,    0,    0]    [SADD R3  ZERO  ZERO, SADD R3  ZERO  ZERO, SADD R3  ZERO  ZERO, SADD R3  ZERO  ZERO ]    
[   0,    0,    0,    0]    [SADD R3  ZERO  ZERO

In [5]:
from cgra import *
from kernels import *
kernel_name = "mmul_os"
# ----------- Op 4 - v3 ---------
# Input data
# 8x4x8
ROWS_C = 8
COLS_C = 8
COLS_A = 2
# Clear memory
kernel_clear_memory(kernel_name)
# Add A
A = list(range(1, 8*4+1, 1))
baseA = 5000
kernel_add_memory_region(kernel_name, baseA, A)
# Add B
B = list(range(1, 4*8+1, 1))
baseB = 2000
kernel_add_memory_region(kernel_name, baseB, B)
# Add C
baseC = 10000
# Add auxiliar address and counters
# &B[0][0], nItLoopColsC, &A[0][0], &C[0][3], &C[1][0], B[0][1], nItLoopColsA, &A[1][0], ...
nItLoopColsC = int(COLS_C/4)
nItLoopColsA = int(COLS_A)
nItLoopRowsC = int(ROWS_C/4)
baseAux = 0
auxiliar_values = [baseB, nItLoopColsC, baseA, baseC+3*4,
                   baseC+4*COLS_C, baseB+4, nItLoopColsA, baseA+COLS_A*4,
                   baseA+COLS_A*2*4, baseC+4+4*2*COLS_C, baseB+4*2, 0,
                   nItLoopRowsC, baseA+3*4*COLS_A, baseC+8+4*3*COLS_C, baseB+12]
kernel_add_memory_region(kernel_name, baseAux, auxiliar_values)
# Set load and write addrs
load_addrs = [baseAux, baseAux+4*4, baseAux+4*4*2, baseAux+4*4*3]
store_addrs = [baseC+3*4, baseC+4*COLS_C, baseC+4+2*4*COLS_C, baseC+8+3*4*COLS_C]
# Run the kernel
run(kernel_name, pr=["ROUT","INST"], load_addrs=load_addrs, store_addrs=store_addrs)

Instr =  0 ( 0 )
[   0, 10032, 5032,    2]    [LWD R2  4, LWD R2  4, LWD R2  4, LWD R1  4]    
[   2, 2004, 10068, 5048]    [LWD R1  4, LWD R2  4, LWD R2  4, LWD R2  4]    
[5000,    4, 2008, 10104]    [LWD R2  4, LWD R1  4, LWD R2  4, LWD R2  4]    
[10012, 5016,    0, 2012]    [LWD R2  4, LWD R2  4, LWD R2  4, LWD R2  4]    
-------
Instr =  1 ( 1 )
[   0, 10032, 5032,    0]    [NOP , NOP , NOP , SADD R2  ZERO  ZERO]    
[   0, 2004, 10068, 5048]    [SADD R2  ZERO  ZERO, NOP , NOP , NOP ]    
[5000,    0, 2008, 10104]    [NOP , SADD R2  ZERO  ZERO, NOP , NOP ]    
[10012, 5016,    0, 2012]    [NOP , NOP , NOP , NOP ]    
-------
Instr =  2 ( 2 )
[   0, 10032, 5032,   28]    [NOP , NOP , NOP , BEQ R2  R1  28]    
[   0, 2004, 10068, 5048]    [NOP , NOP , NOP , NOP ]    
[5000,    0, 2008, 10104]    [NOP , NOP , NOP , NOP ]    
[10012, 5016,    0, 2012]    [NOP , NOP , NOP , NOP ]    
-------
Instr =  3 ( 3 )
[   0, 10032, 5032,   28]    [NOP , NOP , NOP , NOP ]    
[  24, 2004, 10068,